In [1]:
import boto3
# print(f"SageMaker Version: {sagemaker.__version__}")
# print(f"Installation Path: {sagemaker.__file__}")
import sagemaker
from sagemaker.pytorch import PyTorch

from botocore.exceptions import ClientError


# Get the caller identity
sts_client = boto3.client('sts')
identity = sts_client.get_caller_identity()
arn = identity['Arn']
username = arn.split('/')[-1]
import botocore


print(f"Account: {identity['Account']}")
print(f"UserId: {identity['UserId']}")
print(f"ARN: {identity['Arn']}")
print(f"Your AWS Username is: {username}")


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/amannindra/Library/Application Support/sagemaker/config.yaml
Account: 253490779227
UserId: AIDATWBJ2KRN6C42QYYWF
ARN: arn:aws:iam::253490779227:user/amannindra2
Your AWS Username is: amannindra2


In [ ]:
# Stop a specific job
sm_client = boto3.client('sagemaker', region_name='us-west-1')
training_job_name = 'pytorch-training-2025-11-25-02-50-20-903'


start
An error occurred: Requested resource not found.


In [10]:

# REPLACE THIS STRING with the actual Role ARN you created in IAM
role = "arn:aws:iam::253490779227:role/service-role/AmazonSageMakerAdminIAMExecutionRole" 
boto_session = boto3.Session(region_name='us-west-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)

print(f"SageMaker Session Region: {sagemaker_session.boto_region_name}")
print(f"Default Bucket: {sagemaker_session.default_bucket()}")


SageMaker Session Region: us-west-1
Default Bucket: sagemaker-us-west-1-253490779227


In [ ]:
s3_input_data = 's3://animal-classification-dss/' 

# Configure boto3 with retry logic for large uploads
from botocore.config import Config

boto_config = Config(
    retries={
        'max_attempts': 10,
        'mode': 'adaptive'
    },
    connect_timeout=60,
    read_timeout=120
)

# Recreate session with retry config
boto_session = boto3.Session(region_name='us-west-1')
sagemaker_session = sagemaker.Session(
    boto_session=boto_session,
    default_bucket='sagemaker-us-west-1-253490779227'
)

estimator = PyTorch(
    entry_point="simple_train.py",          # The script we created above
    source_dir=".",                  # Uploads the current directory (with train.py) to AWS
    # dependencies=['requirements_sagemaker.txt'],  # Use minimal requirements
    role=role,                       # Your IAM Role ARN
    framework_version='2.1',         # PyTorch version
    py_version='py310',              # Python version
    instance_count=1,
    instance_type='ml.g4dn.xlarge',     # Use 'ml.g4dn.xlarge' for GPU training
    hyperparameters={
        'epochs': 5,
        'batch-size': 32
    },
    sagemaker_session=sagemaker_session
)
estimator.fit({'training': s3_input_data})


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-12-19-20-27-45-421


2025-12-19 20:30:12 Starting - Starting the training job
2025-12-19 20:30:12 Pending - Training job waiting for capacity...
2025-12-19 20:30:25 Pending - Preparing the instances for training...
2025-12-19 20:30:50 Downloading - Downloading input data...
2025-12-19 20:31:31 Downloading - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
2025-12-19 20:35:23

In [13]:
model_name = 'animal-classification-resnet18-model1'  # Your custom model name
model = estimator.create_model(
    model_data=estimator.model_data,  # Uses the model artifacts from training
    name=model_name,
    role=role,
    image_uri=estimator.training_image_uri()  # Uses the same image as training
)

print(f"Model '{model_name}' created successfully!")
print(f"Model data location: {estimator.model_data}")
print(f"Model object: {model}")


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 model_name = 'animal-classification-resnet18-model1'  # Your custom model name              │
│ ❱  2 model = estimator.create_model(                                                             │
│    3 │   model_data=estimator.model_data,  # Uses the model artifacts from training              │
│    4 │   name=model_name,                                                                        │
│    5 │   role=role,                                                                              │
│                                                                                                  │
│ /Users/amannindra/Projects/DSS-Image-Classification/.venv/lib/python3.11/site-packages/sagemaker │
│ /pytorch/estimator.py:809 in create_model                                                        │
│                                                                                                  │
│    806 │   │                                                                                     │
│    807 │   │   kwargs["name"] = self._get_or_create_name(kwargs.get("name"))                     │
│    808 │   │                                                                                     │
│ ❱  809 │   │   return PyTorchModel(                                                              │
│    810 │   │   │   self.model_data,                                                              │
│    811 │   │   │   role or self.role,                                                            │
│    812 │   │   │   entry_point or self._model_entry_point(),                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: PyTorchModel.__init__() got multiple values for argument 'model_data'

In [9]:


# Initialize S3 client
s3_client = boto3.client('s3', region_name='us-west-1')

# Or initialize S3 resource (higher-level interface)
s3_resource = boto3.resource('s3', region_name='us-west-1')

# Your bucket name
bucket_name = 'animal-classification-dss'

# Example 1: List objects in the bucket
# try:
#     response = s3_client.list_objects_v2(Bucket=bucket_name)
#     if 'Contents' in response:
#         print("Objects in bucket:")
#         for obj in response['Contents']:
#             print(f"  {obj['Key']} - Size: {obj['Size']} bytes")
#     else:
#         print("Bucket is empty")
# except ClientError as e:
#     print(f"Error accessing bucket: {e}")

# # Example 2: Upload a file
# def upload_file(file_path, object_key):
#     try:
#         s3_client.upload_file(file_path, bucket_name, object_key)
#         print(f"File {file_path} uploaded as {object_key}")
#     except ClientError as e:
#         print(f"Error uploading file: {e}")

# # Example 3: Download a file
# def download_file(object_key, local_file_path):
#     try:
#         s3_client.download_file(bucket_name, object_key, local_file_path)
#         print(f"File {object_key} downloaded to {local_file_path}")
#     except ClientError as e:
#         print(f"Error downloading file: {e}")

# Example 4: Using S3 resource (alternative approach)
bucket = s3_resource.Bucket(bucket_name)

print(bucket)

# List objects using resource


# for obj in bucket.objects.all():
#     print(f"Object: {obj.key}")

# Upload using resource
# bucket.upload_file('local_file.txt', 'remote_file.txt')

# Download using resource
# bucket.download_file('remote_file.txt', 'local_file.txt')


s3.Bucket(name='animal-classification-dss')


In [ ]:
# After training completes, create a named model
# Method 1: Using estimator.create_model() (Recommended)
# This uses the training job artifacts automatically

model_name = 'animal-classification-resnet18-model'  # Your custom model name
model = estimator.create_model(
    model_data=estimator.model_data,  # Uses the model artifacts from training
    name=model_name,
    role=role,
    image_uri=estimator.training_image_uri()  # Uses the same image as training
)

print(f"Model '{model_name}' created successfully!")
print(f"Model data location: {estimator.model_data}")
print(f"Model object: {model}")

# Alternative Method 2: Using boto3 API directly (if you need more control)
# Uncomment below if you prefer this approach:

# from datetime import datetime
# sm_client = boto3.client('sagemaker', region_name='us-west-1')
# 
# model_name = f'animal-classification-resnet18-{datetime.now().strftime("%Y%m%d-%H%M%S")}'
# 
# # Get the model data from the training job
# training_job_name = estimator.latest_training_job.name
# training_job_info = sm_client.describe_training_job(TrainingJobName=training_job_name)
# model_data_url = training_job_info['ModelArtifacts']['S3ModelArtifacts']
# 
# # Create the model
# container = {
#     'Image': estimator.training_image_uri(),
#     'ModelDataUrl': model_data_url
# }
# 
# response = sm_client.create_model(
#     ModelName=model_name,
#     ExecutionRoleArn=role,
#     PrimaryContainer=container
# )
# 
# print(f"Model '{model_name}' created successfully!")
# print(f"Model ARN: {response['ModelArn']}")
